<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uncomment the code below if you are running on google colab
If you are running the code on the google colab then you must have the data set on your google drive. Link to the dataset where you can download it form https://drive.google.com/file/d/1CAK0Cj1kNgoZ0FRxwwCflbh-1-hKDPoK/view?usp=sharing



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [2]:
#!unzip /content/drive/MyDrive/DL/data.zip # Comment out for niranjan's
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip #comment out for Angel's



Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [3]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [5]:
data = pd.concat([data_train, data_test, data_validation ])
df = data

In [6]:
new_df=df.sample(25000)

In [7]:
new_df.head(3)

,id,article,highlights
101730,0f1c9ead730b4eee665f24ea838508f77dc2f4a4,"By . Mark Duell . PUBLISHED: . 01:14 EST, 1 February 2013 . | . UPDATED: . 15:21 EST, 5 February 2013 . Using nothing but his hands and a rake, this artist's creations look amazing - until the tid...",Tony Plant draws sand art along Cornish coast before it's washed away .\nFifty-year-old artist created these pieces at three beaches in Newquay .\nFather of two has spent 20 years working on this ...
242122,c55cdb9923e14892cae7bfbe9c10a887c591076c,Uruguay may believe Luis Suarez will be fit for their first World Cup game but a three-week turnaround is pushing it for a meniscus injury. The meniscus is a piece of cartilage in the knee that ac...,Uruguay believe Luis Suarez will be fit for their first World Cup 2014 match against Costa Rica .\nLiverpool striker picked up a meniscus injury in training on Wednesday .\nDespite undergoing surg...
215309,a2b8eb18b57aeb5181906f8883812495de291798,The South Africa squad looked in a relaxed mood as the players were put through their final paces on Friday ahead of the autumn international against Ireland. The Springboks take on Joseph Schmid...,South Africa face Ireland at the Aviva Stadium on Saturday .\nThe Springboks trained at the venue on Friday and looked in relaxed mood .\nHead coach Heyneke Meyer has picked the same starting XV t...


In [8]:
new_df.shape

(25000, 3)

In [9]:
sample=new_df.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
["By . Daily Mail Reporter . Holly Hewett from Kent, whose mobile phone caught fire while it was on charge . A woman narrowly escaped serious injury after she woke up to find her mobile phone had caught fire while charging. Holly Hewett from Kelmsley in Kent was woken by a ‘sizzling sound’ and the smell of burning. She looked down and saw her Samsung Galaxy S4 smouldering as it charged on the carpeted floor. At the risk of being burnt, she grabbed the smartphone, ran to the bathroom and threw it in the sink. The 25-year-old said: ‘I’ve never had anything like this happen to me before. ‘It had only been charging for about 50 minutes. Even though it was dark I could smell the smoke. ‘I picked it up and the screen sparked. As I walked to the bathroom the battery fell out of the casing at the back onto the floor. I didn’t burn myself but there’s a hole in the carpet where it scorched it.’ Miss Hewett then grabbed the smartphone and ran to the bathroom and threw it in the si

Checking for Null values


In [10]:
new_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [11]:
new_df.duplicated(subset=['article','highlights']).sum()

19

In [12]:
new_df=new_df.drop_duplicates(subset=['article','highlights'])
new_df.shape

(24981, 3)

In [13]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [14]:
new_df['article'][:10]

101730    By . Mark Duell . PUBLISHED: . 01:14 EST, 1 February 2013 . | . UPDATED: . 15:21 EST, 5 February 2013 . Using nothing but his hands and a rake, this artist's creations look amazing - until the tid...
242122    Uruguay may believe Luis Suarez will be fit for their first World Cup game but a three-week turnaround is pushing it for a meniscus injury. The meniscus is a piece of cartilage in the knee that ac...
215309    The South Africa squad looked in a relaxed mood as the players were put through their  final paces on Friday ahead of the autumn international against Ireland. The Springboks take on Joseph Schmid...
159728    With his speech in front of the United Nations today, Leonardo DiCaprio cemented his reputation as one of the world's highest-profile activists on climate change. 'You can make history ...or be vi...
98124     This cheeky sea lion was clearly bored of the sea and decided to wander into an open-air swimming pool instead. The brazen animal passed through the r

In [15]:
new_df['article'] = new_df['article'].str.lower()
new_df['highlights'] = new_df['highlights'].str.lower()

new_df.head()

,id,article,highlights
101730,0f1c9ead730b4eee665f24ea838508f77dc2f4a4,"by . mark duell . published: . 01:14 est, 1 february 2013 . | . updated: . 15:21 est, 5 february 2013 . using nothing but his hands and a rake, this artist's creations look amazing - until the tid...",tony plant draws sand art along cornish coast before it's washed away .\nfifty-year-old artist created these pieces at three beaches in newquay .\nfather of two has spent 20 years working on this ...
242122,c55cdb9923e14892cae7bfbe9c10a887c591076c,uruguay may believe luis suarez will be fit for their first world cup game but a three-week turnaround is pushing it for a meniscus injury. the meniscus is a piece of cartilage in the knee that ac...,uruguay believe luis suarez will be fit for their first world cup 2014 match against costa rica .\nliverpool striker picked up a meniscus injury in training on wednesday .\ndespite undergoing surg...
215309,a2b8eb18b57aeb5181906f8883812495de291798,the south africa squad looked in a relaxed mood as the players were put through their final paces on friday ahead of the autumn international against ireland. the springboks take on joseph schmid...,south africa face ireland at the aviva stadium on saturday .\nthe springboks trained at the venue on friday and looked in relaxed mood .\nhead coach heyneke meyer has picked the same starting xv t...
159728,5a7995dda352041a21fce306c3b0f67ce57e6c10,"with his speech in front of the united nations today, leonardo dicaprio cemented his reputation as one of the world's highest-profile activists on climate change. 'you can make history ...or be vi...",leonardo dicaprio has positioned himself as one of the highest-profile climate change activists in the world .\nhe told the united nations today: 'you can make history ...or be vilified by it'\ndi...
98124,0a55dc62e92ab43e7ba33178024a2e96bffc446f,this cheeky sea lion was clearly bored of the sea and decided to wander into an open-air swimming pool instead. the brazen animal passed through the reception area and a cafe before plunging into ...,"animal wandered through the reception as a pool in dunedin, new zealand .\nsunned itself for a bit, before taking the plunge and doing some laps .\nswimmers made quick exit out of complex until se..."


In [16]:
new_df['article']=new_df['article'].str.lower()

new_df['highlights']=new_df['highlights'].str.lower()

Converting to lower case

In [17]:
new_df.head()

,id,article,highlights
101730,0f1c9ead730b4eee665f24ea838508f77dc2f4a4,"by . mark duell . published: . 01:14 est, 1 february 2013 . | . updated: . 15:21 est, 5 february 2013 . using nothing but his hands and a rake, this artist's creations look amazing - until the tid...",tony plant draws sand art along cornish coast before it's washed away .\nfifty-year-old artist created these pieces at three beaches in newquay .\nfather of two has spent 20 years working on this ...
242122,c55cdb9923e14892cae7bfbe9c10a887c591076c,uruguay may believe luis suarez will be fit for their first world cup game but a three-week turnaround is pushing it for a meniscus injury. the meniscus is a piece of cartilage in the knee that ac...,uruguay believe luis suarez will be fit for their first world cup 2014 match against costa rica .\nliverpool striker picked up a meniscus injury in training on wednesday .\ndespite undergoing surg...
215309,a2b8eb18b57aeb5181906f8883812495de291798,the south africa squad looked in a relaxed mood as the players were put through their final paces on friday ahead of the autumn international against ireland. the springboks take on joseph schmid...,south africa face ireland at the aviva stadium on saturday .\nthe springboks trained at the venue on friday and looked in relaxed mood .\nhead coach heyneke meyer has picked the same starting xv t...
159728,5a7995dda352041a21fce306c3b0f67ce57e6c10,"with his speech in front of the united nations today, leonardo dicaprio cemented his reputation as one of the world's highest-profile activists on climate change. 'you can make history ...or be vi...",leonardo dicaprio has positioned himself as one of the highest-profile climate change activists in the world .\nhe told the united nations today: 'you can make history ...or be vilified by it'\ndi...
98124,0a55dc62e92ab43e7ba33178024a2e96bffc446f,this cheeky sea lion was clearly bored of the sea and decided to wander into an open-air swimming pool instead. the brazen animal passed through the reception area and a cafe before plunging into ...,"animal wandered through the reception as a pool in dunedin, new zealand .\nsunned itself for a bit, before taking the plunge and doing some laps .\nswimmers made quick exit out of complex until se..."


In [18]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [19]:
new_df['article'] = new_df['article'].apply(remove_html_tags)

new_df['highlights'] = new_df['highlights'].apply(remove_html_tags)

In [20]:
new_df.head()

,id,article,highlights
101730,0f1c9ead730b4eee665f24ea838508f77dc2f4a4,"by . mark duell . published: . 01:14 est, 1 february 2013 . | . updated: . 15:21 est, 5 february 2013 . using nothing but his hands and a rake, this artist's creations look amazing - until the tid...",tony plant draws sand art along cornish coast before it's washed away .\nfifty-year-old artist created these pieces at three beaches in newquay .\nfather of two has spent 20 years working on this ...
242122,c55cdb9923e14892cae7bfbe9c10a887c591076c,uruguay may believe luis suarez will be fit for their first world cup game but a three-week turnaround is pushing it for a meniscus injury. the meniscus is a piece of cartilage in the knee that ac...,uruguay believe luis suarez will be fit for their first world cup 2014 match against costa rica .\nliverpool striker picked up a meniscus injury in training on wednesday .\ndespite undergoing surg...
215309,a2b8eb18b57aeb5181906f8883812495de291798,the south africa squad looked in a relaxed mood as the players were put through their final paces on friday ahead of the autumn international against ireland. the springboks take on joseph schmid...,south africa face ireland at the aviva stadium on saturday .\nthe springboks trained at the venue on friday and looked in relaxed mood .\nhead coach heyneke meyer has picked the same starting xv t...
159728,5a7995dda352041a21fce306c3b0f67ce57e6c10,"with his speech in front of the united nations today, leonardo dicaprio cemented his reputation as one of the world's highest-profile activists on climate change. 'you can make history ...or be vi...",leonardo dicaprio has positioned himself as one of the highest-profile climate change activists in the world .\nhe told the united nations today: 'you can make history ...or be vilified by it'\ndi...
98124,0a55dc62e92ab43e7ba33178024a2e96bffc446f,this cheeky sea lion was clearly bored of the sea and decided to wander into an open-air swimming pool instead. the brazen animal passed through the reception area and a cafe before plunging into ...,"animal wandered through the reception as a pool in dunedin, new zealand .\nsunned itself for a bit, before taking the plunge and doing some laps .\nswimmers made quick exit out of complex until se..."


In [21]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [22]:
new_df['article'] = new_df['article'].apply(remove_punc)

new_df['highlights'] = new_df['highlights'].apply(remove_punc)

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_df['article'] = new_df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
new_df['highlights'] = new_df['highlights'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
data_train.head(1)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown..."


In [26]:
new_df.head(1)

,id,article,highlights
101730,0f1c9ead730b4eee665f24ea838508f77dc2f4a4,mark duell published 0114 est 1 february 2013 updated 1521 est 5 february 2013 using nothing hands rake artists creations look amazing tide comes tony plant 50 newquay cornwall drew eyecatching pi...,tony plant draws sand art along cornish coast washed away fiftyyearold artist created pieces three beaches newquay father two spent 20 years working kind temporary art


In [27]:
#!pip install contractions


In [27]:
max_len_text=30
max_len_summary=10

In [28]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(df['article'],df['highlights'],test_size=0.1,random_state=0,shuffle=True)

Three different methods of tokenization:-
Choose only one of them best on the speed

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1



In [ ]:
import tensorflow as tf

hub_url = "https://tfhub.dev/tensorflow/text/en/tokenizer/2"
x_tokenizer = tf.hub.load(hub_url)

x_tr = x_tokenizer(x_tr)
x_val = x_tokenizer(x_val)

x_tr = tf.sequence.pad(x_tr, maxlen=max_len_text)
x_val = tf.sequence.pad(x_val, maxlen=max_len_text)

x_voc_size = len(x_tokenizer.vocab)

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', oov_token='<UNK>')
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post', truncating='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post', truncating='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

End of the tokenization method for x and begining of the tokenization of y
for y only one method is used so far and based on the performance of the x tokenization. One of the method is used.

In [31]:
#preparing a tokenizer for summary on training data
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr)
y_val   =   y_tokenizer.texts_to_sequences(y_val)

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

LSTM model building

In [32]:
from keras import backend as K
K.clear_session()
latent_dim = 300
embedding_dim=100 #Keeping latent dimension to 300 and introducing embedding dimension of 100 to reduce the number of parameters of the model
#condider chaging the latent_dim to smaller value such as 100 or 300
#also adding a seperate embedding_dim can help to reduce the compleixity of the model

In [33]:
#encoder
encoder_inputs = Input(shape=(max_len_text,))
enc_emb = Embedding(x_voc_size, embedding_dim,trainable=True)(encoder_inputs)

In [34]:
#LSTM 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#LSTM2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

In [35]:
#decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

In [36]:
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])


In [37]:
#pip install attention_keras
import tensorflow as tf
from tensorflow.python.keras import backend as K

logger = tf.get_logger()

class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs

        logger.debug(f"encoder_out_seq.shape = {encoder_out_seq.shape}")
        logger.debug(f"decoder_out_seq.shape = {decoder_out_seq.shape}")

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            logger.debug("Running energy computation step")

            if not isinstance(states, (list, tuple)):
                raise TypeError(f"States must be an iterable. Got {states} of type {type(states)}")

            encoder_full_seq = states[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_full_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim

            logger.debug(f"U_a_dot_h.shape = {U_a_dot_h.shape}")

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)

            logger.debug(f"Ws_plus_Uh.shape = {Ws_plus_Uh.shape}")

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            logger.debug(f"ei.shape = {e_i.shape}")

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            logger.debug("Running attention vector computation step")

            if not isinstance(states, (list, tuple)):
                raise TypeError(f"States must be an iterable. Got {states} of type {type(states)}")

            encoder_full_seq = states[-1]

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_full_seq * K.expand_dims(inputs, -1), axis=1)

            logger.debug(f"ci.shape = {c_i.shape}")

            return c_i, [c_i]

        # we don't maintain states between steps when computing attention
        # attention is stateless, so we're passing a fake state for RNN step function
        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e], constants=[encoder_out_seq]
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c], constants=[encoder_out_seq]
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [38]:
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

In [39]:
# Concat attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

In [40]:
#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

In [41]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 100)      79060900    ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 30, 100),    80400       ['embedding[0][0]']              
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                              

In [42]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy') #using Adam optimizer instead of rms for better performance and efficieny in terms of speed

In [43]:
#using Early stoppage to stop the training of the model once the validation increasers

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=20,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/20
  52/2194 [..............................] - ETA: 15:12:31 - loss: 10.7689

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

In [ ]:
# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

In [ ]:
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

In [ ]:
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])



In [ ]:
#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

In [ ]:
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

In [ ]:
# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
            if (sampled_token == 'end' or len(decoded_sentence.split()) >= (max_len_summary-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
for i in range(len(x_val)):
  print("Review:",seq2text(x_val[i]))
  print("Original summary:",seq2summary(y_val[i]))
  print("Predicted summary:",decode_sequence(x_val[i].reshape(1,max_len_text)))
  print("\n")